<a href="https://colab.research.google.com/github/yzhan480/CS535_Final_Project/blob/main/CS535Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os
import time
from torch.utils.data import Dataset, DataLoader

In [3]:
training_data = pd.read_pickle(r'/content/drive/My Drive/training_set.pkl')
testing_data_1 = pd.read_pickle(r'/content/drive/My Drive/testing_set1.pkl')
testing_data_2 = pd.read_pickle(r'/content/drive/My Drive/testing_set2.pkl')

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
len(training_data)

2000

In [6]:
training_data[0]

,Open,High,Low,Close,Volume
0,0.672106,0.679902,0.669265,0.671966,0.336899
1,0.672106,0.674597,0.670546,0.672106,0.208023
2,0.672106,0.678621,0.671827,0.676282,0.142405
3,0.676338,0.677340,0.672384,0.673498,0.146400
4,0.673442,0.676004,0.672997,0.675892,0.116960
...,...,...,...,...,...
2197,0.680849,0.683633,0.672997,0.674055,0.351048
2198,0.674389,0.674890,0.671326,0.674055,0.143069
2199,0.674110,0.676227,0.672440,0.676004,0.312048
2200,0.675725,0.675837,0.668820,0.669321,0.279230


## Task 1: Adding label

In [7]:
label = []
for stock in range(2000):
  label.append([])

I think in each stock row 1 to 2201 has a label respectively except row 0.

In [8]:
for stock in range(2000):
  a = 0
  b = 0
  c = 0
  percent_change = []
  for row in range(2201):
    percentage_change = (training_data[stock]['Close'][row+1]-training_data[stock]['Close'][row])/(training_data[stock]['Close'][row])
    percent_change.append(percentage_change)
  
  percent_change_origin = percent_change.copy()
  percent_change.sort()

  
  for row in range(2201):
    if percent_change_origin[row] < percent_change[733]:
      label[stock].append([1,0,0])
      a += 1
    if percent_change_origin[row] >= percent_change[733] and percent_change_origin[row] < percent_change[1467]:
      label[stock].append([0,1,0])
      b += 1
    if percent_change_origin[row] >= percent_change[1467]:
      label[stock].append([0,0,1])
      c += 1

a,b,c # for verification

(733, 734, 734)

Here, lable is a list. And each element of the label is also a list which contains the labels of corresponding stock. So len(label) is 2000, len(label[i]) is 2201. And label[i][j] is a label for ith stock jth day because day 2201 does not have a label. And one label is like (0,1,0) one-hot vector which represents this day belongs to second class "no big change". And first class is "decrease", third class is "increase".

## Task 2: Feature Engineering

Add features

In [9]:
for stock in range(2000):
  training_data[stock] = training_data[stock].assign(F1 = training_data[stock].Close *1.1 )
  training_data[stock] = training_data[stock].assign(F2 = training_data[stock].Close *1.2 )
  training_data[stock] = training_data[stock].assign(F3 = training_data[stock].Close *1.3 )
  training_data[stock] = training_data[stock].assign(F4 = training_data[stock].Close *1.4 )
  training_data[stock] = training_data[stock].assign(F5 = training_data[stock].Close *1.5 )
  training_data[stock] = training_data[stock].assign(F6 = training_data[stock].Close ** 2 )
  training_data[stock] = training_data[stock].assign(F7 = training_data[stock].Close ** 3 )
  training_data[stock] = training_data[stock].assign(F8 = training_data[stock].Close ** 4 * 1.5)
  training_data[stock] = training_data[stock].assign(F9 = np.sqrt(np.abs(training_data[stock].Close)))
  training_data[stock] = training_data[stock].assign(F10 = training_data[stock].Close * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F11 = (training_data[stock].Close **2) * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F12 = training_data[stock].Close * training_data[stock].High)
  training_data[stock] = training_data[stock].assign(F13 = training_data[stock].Close * training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F14 = training_data[stock].Close * training_data[stock].Volume)
  training_data[stock] = training_data[stock].assign(F15 = training_data[stock].Close * training_data[stock].Open**2)
  training_data[stock] = training_data[stock].assign(F16 = training_data[stock].Close * training_data[stock].High**2)
  training_data[stock] = training_data[stock].assign(F17 = training_data[stock].Close * training_data[stock].Low**2)
  training_data[stock] = training_data[stock].assign(F18 = training_data[stock].Close * training_data[stock].Volume**2)
  training_data[stock] = training_data[stock].assign(F19 = (training_data[stock].Close - training_data[stock].Open)/training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F20 = (training_data[stock].Open - training_data[stock].Close)/training_data[stock].Close)
  training_data[stock] = training_data[stock].assign(F21 = (training_data[stock].High - training_data[stock].Low)/training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F22 = (training_data[stock].Low - training_data[stock].High)/training_data[stock].High)
  
  percent_change = []
  for row in range(2201):
    percentage_change = (training_data[stock]['Close'][row+1]-training_data[stock]['Close'][row])/(training_data[stock]['Close'][row])
    percent_change.append(percentage_change)  
  percent_change.append(0.001) #initialize for row 2201
  percent_change = np.array(percent_change)
  
  training_data[stock] = training_data[stock].assign(F23 = percent_change)
  training_data[stock] = training_data[stock].assign(F24 = percent_change*10)
  training_data[stock] = training_data[stock].assign(F25 = percent_change*100)
  training_data[stock] = training_data[stock].assign(F26 = (percent_change**2) * 100)
  training_data[stock] = training_data[stock].assign(F27 = (percent_change**2) * 1000)
  training_data[stock] = training_data[stock].assign(F28 = (percent_change**2) * 10000)
  training_data[stock] = training_data[stock].assign(F29 = (percent_change**3) * 100000)
  training_data[stock] = training_data[stock].assign(F30 = 10*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F31 = 100*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F32 = 10*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F33 = 100*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F34 = 10*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F35 = 100*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F36 = 10*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F37 = 100*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F38 = training_data[stock].Close ** 4 * 5)
  training_data[stock] = training_data[stock].assign(F39 = training_data[stock].Close ** 2 * 1.2)
  training_data[stock] = training_data[stock].assign(F40 = training_data[stock].Close ** 2 * 1.1)
  training_data[stock] = training_data[stock].assign(F41 = np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F42 = np.sqrt(np.abs(percent_change)))
  training_data[stock] = training_data[stock].assign(F43 = np.sqrt(np.abs(percent_change))*10)
  training_data[stock] = training_data[stock].assign(F44 = np.sqrt(np.abs(percent_change))*20)
  training_data[stock] = training_data[stock].assign(F45 = np.sqrt(np.abs(percent_change))*30)
  training_data[stock] = training_data[stock].assign(F46 = np.sqrt(np.abs(percent_change))*40)
  training_data[stock] = training_data[stock].assign(F47 = percent_change*10 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F48 = percent_change*100 + np.random.rand(2202)/10)
  training_data[stock] = training_data[stock].assign(F49 = percent_change*100 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F50 = np.ones(2202)/1000)


In [10]:
for stock in range(2000):
  training_data[stock] = training_data[stock].assign(F51 = 0.9 * training_data[stock].Close *1.1 )
  training_data[stock] = training_data[stock].assign(F52 = 0.9 * training_data[stock].Close *1.2 )
  training_data[stock] = training_data[stock].assign(F53 = 0.9 * training_data[stock].Close *1.3 )
  training_data[stock] = training_data[stock].assign(F54 = 0.9 * training_data[stock].Close *1.4 )
  training_data[stock] = training_data[stock].assign(F55 = 0.9 * training_data[stock].Close *1.5 )
  training_data[stock] = training_data[stock].assign(F56 = 0.9 * training_data[stock].Close ** 2 )
  training_data[stock] = training_data[stock].assign(F57 = 0.9 * training_data[stock].Close ** 3 )
  training_data[stock] = training_data[stock].assign(F58 = 0.9 * training_data[stock].Close ** 4 * 1.5)
  training_data[stock] = training_data[stock].assign(F59 = 0.9 * np.sqrt(np.abs(training_data[stock].Close)))
  training_data[stock] = training_data[stock].assign(F60 = 0.9 * training_data[stock].Close * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F61 = 0.9 * (training_data[stock].Close **2) * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F62 = 0.9 * training_data[stock].Close * training_data[stock].High)
  training_data[stock] = training_data[stock].assign(F63 = 0.9 * training_data[stock].Close * training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F64 = 0.9 * training_data[stock].Close * training_data[stock].Volume)
  training_data[stock] = training_data[stock].assign(F65 = 0.9 * training_data[stock].Close * training_data[stock].Open**2)
  training_data[stock] = training_data[stock].assign(F66 = 0.9 * training_data[stock].Close * training_data[stock].High**2)
  training_data[stock] = training_data[stock].assign(F67 = 0.9 * training_data[stock].Close * training_data[stock].Low**2)
  training_data[stock] = training_data[stock].assign(F68 = 0.9 * training_data[stock].Close * training_data[stock].Volume**2)
  training_data[stock] = training_data[stock].assign(F69 = 0.9 * (training_data[stock].Close - training_data[stock].Open)/training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F70 = 0.9 * (training_data[stock].Open - training_data[stock].Close)/training_data[stock].Close)
  training_data[stock] = training_data[stock].assign(F71 = 0.9 * (training_data[stock].High - training_data[stock].Low)/training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F72 = 0.9 * (training_data[stock].Low - training_data[stock].High)/training_data[stock].High)
  
  percent_change = []
  for row in range(2201):
    percentage_change = (training_data[stock]['Close'][row+1]-training_data[stock]['Close'][row])/(training_data[stock]['Close'][row])
    percent_change.append(percentage_change)  
  percent_change.append(0.001) #initialize for row 2201
  percent_change = np.array(percent_change)
  
  training_data[stock] = training_data[stock].assign(F73 = 0.9 * percent_change)
  training_data[stock] = training_data[stock].assign(F74 = 0.9 * percent_change*10)
  training_data[stock] = training_data[stock].assign(F75 = 0.9 * percent_change*100)
  training_data[stock] = training_data[stock].assign(F76 = 0.9 * (percent_change**2) * 100)
  training_data[stock] = training_data[stock].assign(F77 = 0.9 * (percent_change**2) * 1000)
  training_data[stock] = training_data[stock].assign(F78 = 0.9 * (percent_change**2) * 10000)
  training_data[stock] = training_data[stock].assign(F79 = 0.9 * (percent_change**3) * 100000)
  training_data[stock] = training_data[stock].assign(F80 = 0.9 * 10*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F81 = 0.9 * 100*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F82 = 0.9 * 10*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F83 = 0.9 * 100*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F84 = 0.9 * 10*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F85 = 0.9 * 100*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F86 = 0.9 * 10*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F87 = 0.9 * 100*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F88 = 0.9 * training_data[stock].Close ** 4 * 5)
  training_data[stock] = training_data[stock].assign(F89 = 0.9 * training_data[stock].Close ** 2 * 1.2)
  training_data[stock] = training_data[stock].assign(F90 = 0.9 * training_data[stock].Close ** 2 * 1.1)
  training_data[stock] = training_data[stock].assign(F91 = 0.9 * np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F92 = 0.9 * np.sqrt(np.abs(percent_change)))
  training_data[stock] = training_data[stock].assign(F93 = 0.9 * np.sqrt(np.abs(percent_change))*10)
  training_data[stock] = training_data[stock].assign(F94 = 0.9 * np.sqrt(np.abs(percent_change))*30)
  training_data[stock] = training_data[stock].assign(F95 = 0.9 * np.sqrt(np.abs(percent_change))*20)
  training_data[stock] = training_data[stock].assign(F96 = 0.9 * np.sqrt(np.abs(percent_change))*40)
  training_data[stock] = training_data[stock].assign(F97 = 0.9 * percent_change*10 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F98 = 0.9 * percent_change*100 + np.random.rand(2202)/10)
  training_data[stock] = training_data[stock].assign(F99 = 0.9 * percent_change*100 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F100 = 0.9 * np.ones(2202)/1000)

In [11]:
for stock in range(2000):
  print(training_data[stock].tail())

流式输出内容被截断，只能显示最后 5000 行内容。
2198  2.744256  1.829504  3.659008 -0.092090 -0.882506 -0.927430  0.0009  
2199  2.727736  1.818490  3.636981 -0.089684 -0.899782 -0.915819  0.0009  
2200  1.239998  0.826665  1.653331 -0.011799 -0.169131 -0.185675  0.0009  
2201  0.853815  0.569210  1.138420  0.015965  0.137793  0.091457  0.0009  

[5 rows x 105 columns]
          Open      High       Low     Close    Volume        F1        F2  \
2197  0.496450  0.500609  0.496450  0.497768  0.071710  0.547545  0.597322   
2198  0.497616  0.497616  0.489349  0.491479  0.036097  0.540627  0.589775   
2199  0.491783  0.493051  0.487016  0.490211  0.035099  0.539232  0.588253   
2200  0.490160  0.491773  0.487726  0.489450  0.070256  0.538395  0.587340   
2201  0.489450  0.491378  0.485697  0.487624  0.088028  0.536387  0.585149   

            F3        F4        F5  ...       F91       F92       F93  \
2197  0.647099  0.696876  0.746652  ...  0.004609  0.101165  1.011649   
2198  0.638923  0.688071  0.737219

## Task 3

In [12]:
label = np.array(label)
type(label), len(label[0])

(numpy.ndarray, 2201)

In [13]:
label[0].shape

(2201, 3)

In [14]:
def prepare_training_data(training_data, label):
  for stock in range(4):
    for i in range(2101):
      df = training_data[stock][i:i+100]
      if stock == 0 and i == 0:
        X_train = torch.from_numpy(df.values).float().reshape(1,100,105)
        y_train = torch.from_numpy(label[stock][i+100]).reshape(1,3)
      elif i < 1700:
        X_train = torch.cat((X_train, torch.from_numpy(df.values).float().reshape(1,100,105)),0)
        y_train = torch.cat((y_train, torch.from_numpy(label[stock][i+100]).reshape(1,3)),0)
      elif stock == 0 and i == 1700:
        X_valid = torch.from_numpy(df.values).float().reshape(1,100,105)
        y_valid = torch.from_numpy(label[stock][i+100]).reshape(1,3)
      else:
        X_valid = torch.cat((X_valid, torch.from_numpy(df.values).float().reshape(1,100,105)),0)
        y_valid = torch.cat((y_valid, torch.from_numpy(label[stock][i+100]).reshape(1,3)),0)

  return X_train, y_train, X_valid, y_valid

In [15]:
X_train, y_train, X_valid, y_valid = prepare_training_data(training_data, label)

In [16]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

(torch.Size([6800, 100, 105]),
 torch.Size([6800, 3]),
 torch.Size([1604, 100, 105]),
 torch.Size([1604, 3]))

In [17]:
y_train = y_train.to(torch.float32)
y_valid = y_valid.to(torch.float32)

In [18]:
class Stocks_Data(Dataset):
  '''custom dataset'''
  def __init__(self, data_X, data_Y, transform=None):
    self.X = data_X.to(device)
    self.Y = data_Y.to(device)
    self.transform = transform

  def __len__(self):
    """Returns the size (the number of samples) of the dataset.
    """
    return len(self.Y)
  
  def __getitem__(self, i):
    """Returns the i-th sample and label and applies any transforms defined.

      Args:
        i (int): The index of sample in the data array to retrieve. 
    """  
    label = self.Y[i]
    sample = self.X[i]
    if self.transform:
        sample = self.transform(sample)

    return sample, label

In [19]:
class Model_1(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_1, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.BatchNorm1d(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.BatchNorm1d(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [20]:
class Model_2(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_2, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.BatchNorm1d(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.BatchNorm1d(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [21]:
class Model_3(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_3, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.BatchNorm1d(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.BatchNorm1d(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_2, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(256, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [22]:
class Model_4(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_4, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [23]:
class Model_5(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_5, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [24]:
class Model_6(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_6, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [25]:
class Model_7(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_7, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [26]:
class Model_8(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_8, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [27]:
class Model_9(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_9, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.ReLU(),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [28]:
class Model_X(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_X, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.ReLU(),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [29]:
a = torch.rand(15,3)
a[1].argmax().type(torch.float).item()


0.0

In [30]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):   
        # Compute prediction and loss
        
        model.train()
        pred = model(X).to(torch.float32)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 5 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    true_positive, pred_positive = 0, 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            #correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
            for i in range(len(y)):
                
                if pred[i].argmax().type(torch.float).item()>1:
                    pred_positive += 1
                    
                if pred[i].argmax().type(torch.float).item() == y[i].argmax().type(torch.float).item():
                    correct += 1
                    if pred[i].argmax().type(torch.float).item()>1:
                        true_positive += 1
                        

    percentage_of_positive_predictions = pred_positive / size
    precision = true_positive / pred_positive
    test_loss /= num_batches
    correct /= size
    print(f"Validation: \n Accuracy: {(100*correct):>0.1f}%, Precision: {(100*precision):>0.1f}%, \
Percentage of positive predictions: {(100*percentage_of_positive_predictions):>0.1f}%, Avg loss: {test_loss:>8f}")

In [31]:
training_data = Stocks_Data(X_train, y_train)
valid_data = Stocks_Data(X_valid, y_valid)

In [32]:
train_dataloader = DataLoader(training_data, batch_size=400, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=400, shuffle=True)

In [33]:
model_1 = Model_1(input_size = 10500, hidden_size_1 = 12500, hidden_size_2 = 6400).to(device)
learning_rate = 1e-3
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_1, loss_fn, optimizer)
    if t > 50:
        test_loop(valid_dataloader, model_1, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, model_1, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.703234  [    0/ 6800]
loss: 0.669393  [ 2000/ 6800]
loss: 0.652274  [ 4000/ 6800]
loss: 0.637390  [ 6000/ 6800]
Validation: 
 Accuracy: 27.2%, Precision: 41.0%, Percentage of positive predictions: 4.9%, Avg loss: 0.773615
Validation: 
 Accuracy: 37.3%, Precision: 41.2%, Percentage of positive predictions: 1.4%, Avg loss: 0.711613
Epoch 2
-------------------------------
loss: 0.631344  [    0/ 6800]
loss: 0.631609  [ 2000/ 6800]
loss: 0.627191  [ 4000/ 6800]
loss: 0.622720  [ 6000/ 6800]
Validation: 
 Accuracy: 32.5%, Precision: 40.0%, Percentage of positive predictions: 26.3%, Avg loss: 0.634656
Validation: 
 Accuracy: 41.2%, Precision: 44.8%, Percentage of positive predictions: 11.4%, Avg loss: 0.623222
Epoch 3
-------------------------------
loss: 0.616711  [    0/ 6800]
loss: 0.615386  [ 2000/ 6800]
loss: 0.598567  [ 4000/ 6800]
loss: 0.611202  [ 6000/ 6800]
Validation: 
 Accuracy: 32.5%, Precision: 38.7%, Percentage of positive predic

In [34]:
model_2 = Model_2(input_size = 10500, hidden_size_1 = 2048, hidden_size_2 = 1024).to(device)
learning_rate = 1e-3
loss_fn = nn.BCELoss() # because of using softmax layer
optimizer = torch.optim.Adam(model_2.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_2, loss_fn, optimizer)
    if t > 50:
        test_loop(valid_dataloader, model_2, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, model_2, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.644364  [    0/ 6800]
loss: 0.631656  [ 2000/ 6800]
loss: 0.633002  [ 4000/ 6800]
loss: 0.619365  [ 6000/ 6800]
Validation: 
 Accuracy: 33.0%, Precision: 36.3%, Percentage of positive predictions: 17.5%, Avg loss: 0.645080
Validation: 
 Accuracy: 42.5%, Precision: 39.7%, Percentage of positive predictions: 16.1%, Avg loss: 0.629262
Epoch 2
-------------------------------
loss: 0.607671  [    0/ 6800]
loss: 0.618586  [ 2000/ 6800]
loss: 0.620706  [ 4000/ 6800]
loss: 0.620712  [ 6000/ 6800]
Validation: 
 Accuracy: 35.0%, Precision: 41.8%, Percentage of positive predictions: 16.7%, Avg loss: 0.638246
Validation: 
 Accuracy: 40.4%, Precision: 45.8%, Percentage of positive predictions: 8.7%, Avg loss: 0.621850
Epoch 3
-------------------------------
loss: 0.603827  [    0/ 6800]
loss: 0.612801  [ 2000/ 6800]
loss: 0.609111  [ 4000/ 6800]
loss: 0.610221  [ 6000/ 6800]
Validation: 
 Accuracy: 36.5%, Precision: 36.5%, Percentage of positive predi

In [35]:
model_3 = Model_3(input_size = 10500, hidden_size_1 = 12500, hidden_size_2 = 6400).to(device)
learning_rate = 1e-3
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_3.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_3, loss_fn, optimizer)
    if t > 50:
        test_loop(valid_dataloader, model_3, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, model_3, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.696343  [    0/ 6800]
loss: 0.679159  [ 2000/ 6800]
loss: 0.659214  [ 4000/ 6800]
loss: 0.652221  [ 6000/ 6800]
Validation: 
 Accuracy: 27.9%, Precision: 33.8%, Percentage of positive predictions: 18.5%, Avg loss: 0.882580
Validation: 
 Accuracy: 37.1%, Precision: 33.3%, Percentage of positive predictions: 13.2%, Avg loss: 0.757095
Epoch 2
-------------------------------
loss: 0.653418  [    0/ 6800]
loss: 0.645887  [ 2000/ 6800]
loss: 0.632174  [ 4000/ 6800]
loss: 0.628089  [ 6000/ 6800]
Validation: 
 Accuracy: 35.2%, Precision: 39.7%, Percentage of positive predictions: 15.4%, Avg loss: 0.636484
Validation: 
 Accuracy: 38.5%, Precision: 38.9%, Percentage of positive predictions: 16.1%, Avg loss: 0.631764
Epoch 3
-------------------------------
loss: 0.631541  [    0/ 6800]
loss: 0.633271  [ 2000/ 6800]
loss: 0.625167  [ 4000/ 6800]
loss: 0.621987  [ 6000/ 6800]
Validation: 
 Accuracy: 34.7%, Precision: 38.2%, Percentage of positive pred

In [39]:
model_4 = Model_4(input_size = 10500, hidden_size_1 = 2048, hidden_size_2 = 1024).to(device)
learning_rate = 1e-3
loss_fn = nn.BCELoss() # because of using softmax layer
optimizer = torch.optim.Adam(model_4.parameters(), lr=learning_rate)

epochs = 80
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_4, loss_fn, optimizer)
    if t > 50:
        test_loop(valid_dataloader, model_4, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, model_4, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.659238  [    0/ 6800]
loss: 0.642554  [ 2000/ 6800]
loss: 0.641937  [ 4000/ 6800]
loss: 0.644483  [ 6000/ 6800]
Epoch 2
-------------------------------
loss: 0.638991  [    0/ 6800]
loss: 0.639229  [ 2000/ 6800]
loss: 0.640667  [ 4000/ 6800]
loss: 0.637422  [ 6000/ 6800]
Epoch 3
-------------------------------
loss: 0.637536  [    0/ 6800]
loss: 0.638147  [ 2000/ 6800]
loss: 0.634189  [ 4000/ 6800]
loss: 0.636190  [ 6000/ 6800]
Epoch 4
-------------------------------
loss: 0.635907  [    0/ 6800]
loss: 0.636078  [ 2000/ 6800]
loss: 0.637344  [ 4000/ 6800]
loss: 0.637809  [ 6000/ 6800]
Epoch 5
-------------------------------
loss: 0.633013  [    0/ 6800]
loss: 0.637300  [ 2000/ 6800]
loss: 0.636515  [ 4000/ 6800]
loss: 0.635085  [ 6000/ 6800]
Epoch 6
-------------------------------
loss: 0.637060  [    0/ 6800]
loss: 0.636335  [ 2000/ 6800]
loss: 0.639795  [ 4000/ 6800]
loss: 0.637538  [ 6000/ 6800]
Epoch 7
-------------------------------


In [ ]:
model_5 = Model_5(input_size = 10500, hidden_size_1 = 12500, hidden_size_2 = 6400).to(device)
learning_rate = 1e-3
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_5.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_5, loss_fn, optimizer)
    test_loop(valid_dataloader, model_5, loss_fn) # validation on validation dataset
    test_loop(train_dataloader, model_5, loss_fn) # validation on training dataset
print("Done!")